# Eve: Making Learing Interesting

## The usage of hidden states

Different with **torch.nn.buffer**, we introduce a new property named **eve.cores.Eve.hidden_state** to record the 
tempory hidden values of a layer. All the hidden states  will be reset while **Eve.reset()** called, and the hidden
states will not be saved along with weights.

In [1]:
import eve
import eve.cores
import torch

class Layer(eve.cores.Eve):
    def __init__(self):
        super().__init__()
        
        # register a hidden states
        self.register_hidden_state("hidden_value_hid", torch.tensor([1.0]))
    
    def forward(self):
        # access the hidden_value just like the attribute of class.
        self.hidden_value_hid += 1


In [2]:
# define L
L = Layer()

# print all hidden states
for k, v in L.named_hidden_states():
    print(k, v.item())

hidden_value_hid 1.0


In [3]:
# update the hidden state
L()

# print the updated hidden state
for k, v in L.named_hidden_states():
    print(k, v.item())

hidden_value_hid 2.0


In [4]:
# clear hidden state to zero
L.reset(set_to_none=False)

for k, v in L.named_hidden_states():
    print(k, v.item())

hidden_value_hid 0.0


In [5]:
# set hidden state to none
L.reset(set_to_none=True)

for k, v in L.named_hidden_states():
    print(k, torch.typename(v))
    # print nothing, the None hidden state will not be fetched.

## The usage of EveParameter

**Eve.EveParameter** is used to control the network arguments which not be counted into the gradient updating.
It will be saved along with weights and have very simily properties with **nn.Parameter**.

Usually, we can use **Eve.EveParameter** to control the evolution of network along time.

In [6]:
import eve
import eve.cores
import torch
import torch.nn as nn
from torch.nn import Parameter
class Layer(eve.cores.Eve):
    def __init__(self):
        super().__init__()
        
        # 0 indicates ReLU, 1 indicates LeakyReLU
        self.register_eve_parameter("fn_flag_eve", Parameter(data=torch.tensor([1.])))
        
        # define the switch operation
        def switch_fn(param, action=None, obs=None):
            if param == 1:
                param.zero_().add_(0)
            elif param == 0:
                param.zero_().add_(1)
        
        self.register_upgrade_fn(self.fn_flag_eve, switch_fn)
    
    def forward(self):
        if self.fn_flag_eve == 0:
            print("using ReLU function")
        elif self.fn_flag_eve == 1:
            print("using LeakyReLU function")        

In [7]:
# define L
L = Layer()

for k, v in L.named_eve_parameters():
    print(k, v.item())

fn_flag_eve 1.0


In [8]:
# check act
L()

using LeakyReLU function


In [9]:
# switch
from eve.cores import upgrade_fn
with torch.no_grad():
    upgrade_fn[L.fn_flag_eve](param=L.fn_flag_eve)

In [10]:
# check act
L()

using ReLU function


In [11]:
# save eve parameters
print(L.state_dict())

OrderedDict([('fn_flag_eve', tensor([0.]))])


## Use Upgrader to control evolution

Like **torch.optim.Optimizer**, we provide **eve.upgrade.Upgrader** to control the evolution of eve paraemters.

In [12]:
# define a upgrader
import eve.upgrade

In [13]:
# add to upgrader
upgrader = eve.upgrade.Upgrader(L.eve_parameters())

print(upgrader.state_dict())

{'state': {}, 'param_groups': [{'params': [0]}]}


In [14]:
# check act
L()

using ReLU function


In [15]:
# upgrade it 
upgrader.step()

In [16]:
# re-check act
L()

using LeakyReLU function
